In [5]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 21 08:51:45 2020

@author: LiuQi
"""

import os
import glob
import numpy as np
from keras import layers, optimizers, models
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.layers import *    
from keras.models import Model,load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Conv2D, MaxPooling2D
import heapq

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [23]:
n = 4

conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = models.Sequential()
model.add(conv_base)
model.add(Flatten())                                                     #平铺层
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(layers.Dense(n, activation='softmax'))

conv_base.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

category = '贵妇'
basic_path = f'E:/jupyter program/Social image/{category}/classify'

In [24]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, )

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
	f'{basic_path}/train',
	target_size=(224, 224),
	batch_size=16,
    
	class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
	f'{basic_path}/validation',
	target_size=(224, 224),
	batch_size=16,
	class_mode='categorical')

Found 1940 images belonging to 4 classes.
Found 487 images belonging to 4 classes.


In [25]:
history = model.fit(
	train_generator,
# 	steps_per_epoch=200,
	epochs=15,
	validation_data=validation_generator,
# 	validation_steps=24,
	verbose=1)

Epoch 1/15
122/122 [==============================] - 214s 2s/step - loss: 2.8934 - acc: 0.6297 - val_loss: 0.9558 - val_acc: 0.7844
Epoch 2/15
122/122 [==============================] - 196s 2s/step - loss: 0.6097 - acc: 0.8353 - val_loss: 0.5254 - val_acc: 0.8686
Epoch 3/15
122/122 [==============================] - 191s 2s/step - loss: 0.4502 - acc: 0.8891 - val_loss: 0.4224 - val_acc: 0.9014
Epoch 4/15
122/122 [==============================] - 186s 2s/step - loss: 0.3983 - acc: 0.9028 - val_loss: 0.4181 - val_acc: 0.9076
Epoch 5/15
122/122 [==============================] - 185s 2s/step - loss: 0.3020 - acc: 0.9168 - val_loss: 0.7038 - val_acc: 0.8994
Epoch 6/15
122/122 [==============================] - 184s 2s/step - loss: 0.3001 - acc: 0.9351 - val_loss: 0.3243 - val_acc: 0.9281
Epoch 7/15
122/122 [==============================] - 184s 2s/step - loss: 0.2984 - acc: 0.9348 - val_loss: 0.6102 - val_acc: 0.8953
Epoch 8/15
122/122 [==============================] - 183s 2s/step - 

In [ ]:
# import matplotlib.pyplot as plt

# def plot_curve(history):
#     acc = history.history['acc']
#     val_acc = history.history['val_acc']
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']
    
#     epochs = range(1,len(acc)+1)
    
#     plt.plot(epochs,acc,'bo',label='Training acc')
#     plt.plot(epochs,val_acc,'b',label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.legend()
    
#     plt.figure()
#     plt.plot(epochs,loss,'bo',label='Training loss')
#     plt.plot(epochs,val_loss,'b',label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()
    
# plot_curve(history)


# test_generator = test_datagen.flow_from_directory(
#     r'C:\Users\liuqi\Desktop\page_segmentation\Unilever\test',
# 	target_size=(224, 224),
# 	batch_size=16,
# 	class_mode='binary')
# test_loss, test_acc = model.evaluate_generator(test_generator, steps=16)
# print('test acc:', test_acc)

In [26]:
# 模型保存
model.save_weights(f'{category}_weight.h5')
model.save(f'{category}.h5')

In [27]:
dic = dict(zip(range(n),(subpath.split('\\')[-1] for subpath in glob.glob(f'{basic_path}/raw image/*'))))

# 模型读取及预测
def append_seg(filename,seg,prob):
    # path = os.path.dirname(filename)
    name = os.path.basename(filename)
    name, ext = os.path.splitext(name)
    # name = f"{name.split('_')[0]}_{name.split('_')[1]}"
    return f"{basic_path}/result/{seg}/{prob}_{name}{ext}"

for subpath in glob.glob(f'{basic_path}/raw image/*'):
    j = subpath.split('\\')[-1]
    if not os.path.exists(f'{basic_path}/result/{j}'):
       os.makedirs(f'{basic_path}/result/{j}') 

In [28]:
model = load_model(f'{category}.h5')
target = f'{basic_path}/use image'
for subpath in glob.glob(f'{target}/*'):
    for im in glob.glob(f'{subpath}/*'):
#         try:
            img = load_img(im,target_size=(224, 224))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            prob = int(heapq.nlargest(1,model.predict(img).tolist()[0])[0]*10)
            segment = np.argmax(model.predict(img), axis=-1)[0]
            new_name = append_seg(im,dic[segment],prob)
            os.rename(im, new_name)
#         except:
#             print(im)

In [15]:
category = '贵妇'
basic_path = f'E:/jupyter program/Social image/{category}/classify'
model = load_model(f'{category}.h5')
target = f'{basic_path}/use image'
for subpath in glob.glob(f'{target}/*'):
    for im in glob.glob(f'{subpath}/*'):
#         try:
            img = load_img(im,target_size=(224, 224))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            prob = int(heapq.nlargest(1,model.predict(img).tolist()[0])[0]*10)
            segment = np.argmax(model.predict(img), axis=-1)[0]
            new_name = append_seg(im,dic[segment],prob)
            os.rename(im, new_name)
#         except:
#             print(im)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1419699857981184839
]


In [4]:
import tensorflow as tf

import timeit



print('version of TF:',tf.__version__)

print(tf.test.is_gpu_available())



gpus = tf.config.list_physical_devices('GPU')

print('GPU devices:')

print(gpus)







version of TF: 2.4.1
False
GPU devices:
[]


In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

Instructions for updating:
non-resource variables are not supported in the long term
Device mapping: no known devices.
[[22. 28.]
 [49. 64.]]
